In [ ]:
import os
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import pickle
import json
import numpy as np

In [ ]:
text_lines = []

for book in os.listdir('./books/'):
    with open(f'./books/{book}', encoding='iso 8859-1') as f:
        text_lines += f.readlines()

text_lines = [a.lower() for a in text_lines]

with open('./text.txt', 'w') as f:
    f.writelines(text_lines)

In [ ]:
# tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(text_lines, target_vocab_size=2**13)

In [ ]:
# with open('tokenizer', 'wb') as f:
#     f.write(pickle.dumps(tokenizer))

In [ ]:
with open('tokenizer', 'rb') as f:
    tokenizer = pickle.loads(f.read())

vocab_size = tokenizer.vocab_size

In [ ]:
text = ''.join(text_lines)

In [ ]:
# tokens = tokenizer.encode(text)

In [ ]:
# with open('tokens.json', 'w') as f:
#     f.write(json.dumps(tokens))

In [ ]:
with open('tokens.json', 'r') as f:
    tokens = json.loads(f.read())

In [44]:
model = keras.Sequential([
    layers.Embedding(input_dim=vocab_size, output_dim=256, input_shape=(100,)),
    layers.LSTM(512),
    layers.Dense(2048),
    layers.Dense(vocab_size, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 100, 256)          2102784   
_________________________________________________________________
lstm_8 (LSTM)                (None, 512)               1574912   
_________________________________________________________________
dense_13 (Dense)             (None, 2048)              1050624   
_________________________________________________________________
dense_14 (Dense)             (None, 8214)              16830486  
Total params: 21,558,806
Trainable params: 21,558,806
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# model.save('model.h5')

In [ ]:
model = keras.models.load_model('model.h5')

In [43]:
for i in range(0, len(tokens), 1024):
    x = []
    y = []

    for ii in range(i, i+1024):
        x += [tokens[ii:(ii+100)]]
        y.append(tokens[ii+100])

    
    x = np.array(x)
    y = keras.utils.to_categorical(y, num_classes=vocab_size)

    model.fit(x, y, batch_size=64)

 2/16 [==>...........................] - ETA: 19s - loss: 6.7608 - accuracy: 0.0234

KeyboardInterrupt: 